# A snippet for directly writing the dataframe "recording" and metadata into BIDS format

Thanks to Yiwei Jia
https://github.com/TheAxonLab/hcph-sops/issues/480

Convert the type of "int64" or "int32" in "metadata" into Type "int" for proper json saving.

In [ ]:
def convert_to_int(metadata):
    if 'CalibrationCount' in metadata:
        metadata['CalibrationCount'] = int(metadata['CalibrationCount']) if isinstance(metadata['CalibrationCount'], (np.int32, np.int64, int)) else metadata['CalibrationCount']
    if "CalibrationLog" in metadata:
        metadata["CalibrationLog"] = [(int(x[0]),x[1]) if isinstance(x[0], (np.int32, np.int64, int)) else x for x in metadata['CalibrationLog']]
    return metadata

convert_metadata = convert_to_int(metadata)

Save BIDS from dataframe "recording" and dict "metadata"

In [ ]:
def write_bids_from_df(
    recording, metadata,
    out_dir,
    filename,
    # exp_run: str | Path,
) -> List[str]:
    """
    Directly save the eye-tracking recording/metadata into a  BIDS structure.

    Parameters
    ----------
    recording : dataframe
        The recording data extracted from the EDF file.
    metadata : dict
        The metadata extracted from the EDF file.
    out_dir : obj:`os.pathlike`
        The path of EDF file. Refers to the folder (not the EDF file).
    filename: str
        The filename of the EDF file. The file name without the suffix, eg: "Subject001"
    
    Returns
    -------
    List[str]
        A list of generated files.

    """

    out_json = out_dir / (filename + ".json")
    out_json.write_text(
        json.dumps(metadata, sort_keys=True, indent=2)
    )

    # Write out data
    out_tsvgz = out_dir / (filename + ".tsv.gz")

    recording.to_csv(
        out_tsvgz,
        sep="\t",
        index=True,
        header=True,
        compression="gzip",
        na_rep="n/a",
    )

    return str(out_tsvgz), str(out_json)

In [ ]:
out_dir = DATA_PATH
# eg: DATA_PATH = Path("\\...\\ET_EDF")
filename = edf_name.split('.')[0]
#eg: filename = 'Subject001'
print(f'bid filename: {filename}')

write_bids_from_df(
    recording, convert_metadata,
    out_dir,
    filename,
)